In [1]:
# Import necessary libraries and modules
import pandas as pd
import numpy as np

# PyMC for Bayesian statistical modeling and probabilistic machine learning
import pymc as pm
# ArviZ for exploratory analysis of Bayesian models
import arviz as az

# Matplotlib and Seaborn for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

from platform import python_version

# Print versions of each package
print(python_version())
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"PyMC version: {pm.__version__}")  # Ensure this matches the PyMC version you're using (pymc3 or pymc4)
print(f"ArviZ version: {az.__version__}")
print(f"matplotlib version: {plt.matplotlib.__version__}")
print(f"seaborn version: {sns.__version__}")

3.9.13
pandas version: 1.4.4
numpy version: 1.23.2
PyMC version: 4.1.7
ArviZ version: 0.12.1
matplotlib version: 3.5.3
seaborn version: 0.12.0


In [2]:
# Read data into DataFrames
df4 = pd.read_csv('../Data/clean/Cohort4Clean.csv')
df5 = pd.read_csv('../Data/clean/Cohort5Clean.csv')
pics = pd.read_csv('../pics/pics.csv')

# Concatenate df4 and df5 to create a combined DataFrame
df = pd.concat([df4,df5])

# Drop duplicate rows based on the 'ID' column
df = df.drop_duplicates(subset=['ID'])

df['EmotionalNumbingZ'] = (df['EmotionalNumbing'] - df['EmotionalNumbing'].mean()) / df['EmotionalNumbing'].std()
df['AgeZ'] = (df['Age'] - df['Age'].mean()) / df['Age'].std()
# Print the shape (number of rows and columns) of the combined and deduplicated DataFrame
print(df.shape)

(1380, 64)


In [3]:
# Transform the DataFrame 'df' from wide to long format using 'melt'
# 'ID', 'PTSD', 'A', 'EmotionalNumbing', and 'PHQ' columns are kept as identifiers
# The values from columns 3 to 43 are reshaped into two columns: 'pic' (variable) and 'rating' (value)
df_long = pd.melt(df, id_vars=['ID','PTSD','AgeZ','EmotionalNumbingZ'], 
                  value_vars=list(df.columns[3:43]),
                  var_name='pic', 
                  value_name='rating')

# Merge 'df_long' with the 'pics' DataFrame on the 'pic' column
# This operation adds information from 'pics' to 'df_long' based on picture names
# The merged DataFrame is then sorted by 'PTSD' and 'ID'
df_long = df_long.merge(pics, left_on='pic', right_on='pic').sort_values(['PTSD','ID'])

# Scale the 'rating' column by dividing each value by 100
df_long['rating'] = df_long['rating']/100

# Create a new column 'scale_rating' by copying values from 'NAPSr' column
df_long['scale_rating'] = df_long['NAPSr']

# Filter the DataFrame to keep only rows where 'scale_rating' is less than 6
df_long = df_long[df_long['scale_rating']<6].reset_index(drop=True)

# Remove rows from 'df_long' where 'rating' has missing (NaN) values
# Store the resulting DataFrame in 'df_long_c'
df_long_c = df_long.dropna(subset='rating')

## Define models 

In [4]:
def pl5_hyper_emotion(df_all, idx, n_sub):
    

    with pm.Model() as five_PL:
        
        # hyper priors
        # Define hyperpriors for group-level parameters
        a1   = pm.TruncatedNormal('a1',   2, 1, lower =  1) # Truncated Normal hyperprior for 'a1'
        a2   = pm.Normal('a2',           10, 1            ) # Normal hyperprior for 'a2'
        mu_b = pm.TruncatedNormal('mu_b', 2, 1, lower =  1) # Truncated Normal hyperprior for mean 'b'
        mu_c = pm.TruncatedNormal('mu_c', 2, 1, lower = .5) # Truncated Normal hyperprior for mean 'c'
        mu_d = pm.TruncatedNormal('mu_d', 2, 1, lower =  0) # Truncated Normal hyperprior for mean 'd'
        mu_g = pm.TruncatedNormal('mu_g', 4, 1, lower =  1) # Truncated Normal hyperprior for mean 'g'
        
        
        # Define priors for parameters of the 5PL regression
        a = pm.Beta('a',              a1, a2,                         shape = n_sub) # Beta distribution for lower asymptote
        b = pm.TruncatedNormal('b', mu_b,  1,  lower = .5,            shape = n_sub) # Truncated Normal distribution for the slope
        c = pm.TruncatedNormal('c', mu_c,  1,  lower = .5, upper = 5, shape = n_sub) # Truncated Normal distribution for mid point
        d = pm.TruncatedNormal('d', mu_d,  1,  lower = .5,            shape = n_sub) # Truncated Normal distribution for upper asymptote
        g = pm.TruncatedNormal('g', mu_g,  1,  lower = .8,            shape = n_sub) # Truncated Normal distribution for asymmetry factor
                
        EN_slope = pm.Normal('EN_slope', 0, 1)
        
        s = b[idx] + EN_slope * df_all['EmotionalNumbingZ']

        # Define prior for standard deviation of observations
        eps = pm.Gamma('eps', 1, 1)

        # Define the 5PL regression model   
        y_hat = d[idx] + ((a[idx]-d[idx])/(1+(df_all['scale_rating'].values/c[idx])**s)**g[idx])

        # Define likelihood function using observed data
        rating = pm.Normal('rating',y_hat, eps, observed=df_all.rating)

        # Sample from the posterior distribution
        trace = pm.sample()#tune = 3000, draws = 2000, target_accept = 0.99)

    return trace

In [5]:
def pl5_hyper_age(df_all, idx, n_sub):
    

    with pm.Model() as five_PL:
        
        # hyper priors
        # Define hyperpriors for group-level parameters
        a1   = pm.TruncatedNormal('a1',   2, 1, lower =  1) # Truncated Normal hyperprior for 'a1'
        a2   = pm.Normal('a2',           10, 1            ) # Normal hyperprior for 'a2'
        mu_b = pm.TruncatedNormal('mu_b', 2, 1, lower =  1) # Truncated Normal hyperprior for mean 'b'
        mu_c = pm.TruncatedNormal('mu_c', 2, 1, lower = .5) # Truncated Normal hyperprior for mean 'c'
        mu_d = pm.TruncatedNormal('mu_d', 2, 1, lower =  0) # Truncated Normal hyperprior for mean 'd'
        mu_g = pm.TruncatedNormal('mu_g', 4, 1, lower =  1) # Truncated Normal hyperprior for mean 'g'
        
        
        # Define priors for parameters of the 5PL regression
        a = pm.Beta('a',              a1, a2,                         shape = n_sub) # Beta distribution for lower asymptote
        b = pm.TruncatedNormal('b', mu_b,  1,  lower = .5,            shape = n_sub) # Truncated Normal distribution for the slope
        c = pm.TruncatedNormal('c', mu_c,  1,  lower = .5, upper = 5, shape = n_sub) # Truncated Normal distribution for mid point
        d = pm.TruncatedNormal('d', mu_d,  1,  lower = .5,            shape = n_sub) # Truncated Normal distribution for upper asymptote
        g = pm.TruncatedNormal('g', mu_g,  1,  lower = .8,            shape = n_sub) # Truncated Normal distribution for asymmetry factor
             
        Age_slope = pm.Normal('Age_slope', 0, 1)
        
        s = b[idx] + (Age_slope * df_all['AgeZ'])

        # Define prior for standard deviation of observations
        eps = pm.Exponential('eps', 1)

        # Define the 5PL regression model   
        y_hat = d[idx] + ((a[idx]-d[idx])/(1+(df_all['scale_rating'].values/c[idx])**s)**g[idx])

        # Define likelihood function using observed data
        rating = pm.Normal('rating',y_hat, eps, observed=df_all.rating)

        trace = pm.sample()#tune = 2000, draws = 2000, target_accept = 0.99)

    return trace

In [6]:
def pl5_hyper_age_en(df_all, idx, n_sub):
    

    with pm.Model() as five_PL:
        
        # hyper priors
        # Define hyperpriors for group-level parameters
        a1   = pm.TruncatedNormal('a1',   2, 1, lower =  1) # Truncated Normal hyperprior for 'a1'
        a2   = pm.Normal('a2',           10, 1            ) # Normal hyperprior for 'a2'
        mu_b = pm.TruncatedNormal('mu_b', 2, 1, lower =  1) # Truncated Normal hyperprior for mean 'b'
        mu_c = pm.TruncatedNormal('mu_c', 2, 1, lower = .5) # Truncated Normal hyperprior for mean 'c'
        mu_d = pm.TruncatedNormal('mu_d', 2, 1, lower =  0) # Truncated Normal hyperprior for mean 'd'
        mu_g = pm.TruncatedNormal('mu_g', 4, 1, lower =  1) # Truncated Normal hyperprior for mean 'g'
        
        
        # Define priors for parameters of the 5PL regression
        a = pm.Beta('a',              a1, a2,                         shape = n_sub) # Beta distribution for lower asymptote
        b = pm.TruncatedNormal('b', mu_b,  1,  lower = .5,            shape = n_sub) # Truncated Normal distribution for the slope
        c = pm.TruncatedNormal('c', mu_c,  1,  lower = .5, upper = 5, shape = n_sub) # Truncated Normal distribution for mid point
        d = pm.TruncatedNormal('d', mu_d,  1,  lower = .5,            shape = n_sub) # Truncated Normal distribution for upper asymptote
        g = pm.TruncatedNormal('g', mu_g,  1,  lower = .8,            shape = n_sub) # Truncated Normal distribution for asymmetry factor
             
        EN_slope = pm.HalfNormal('EN_slope', 1)
        Age_slope = pm.Normal('Age_slope', 0, 1)
        
        s = b[idx] + (EN_slope * df_all['EmotionalNumbingZ'] +
                      Age_slope * df_all['AgeZ'])

        # Define prior for standard deviation of observations
        eps = pm.Exponential('eps', 1)

        # Define the 5PL regression model   
        y_hat = d[idx] + ((a[idx]-d[idx])/(1+(df_all['scale_rating'].values/c[idx])**s)**g[idx])

        # Define likelihood function using observed data
        rating = pm.Normal('rating',y_hat, eps, observed=df_all.rating)

        trace = pm.sample()#tune = 2000, draws = 2000, target_accept = 0.99)

    return trace

In [7]:
def pl5_hyper_inter(df_all, idx, n_sub):
    

    with pm.Model() as five_PL:
        
        # hyper priors
        # Define hyperpriors for group-level parameters
        a1   = pm.TruncatedNormal('a1',   2, 1, lower =  1) # Truncated Normal hyperprior for 'a1'
        a2   = pm.Normal('a2',           10, 1            ) # Normal hyperprior for 'a2'
        mu_b = pm.TruncatedNormal('mu_b', 2, 1, lower =  1) # Truncated Normal hyperprior for mean 'b'
        mu_c = pm.TruncatedNormal('mu_c', 2, 1, lower = .5) # Truncated Normal hyperprior for mean 'c'
        mu_d = pm.TruncatedNormal('mu_d', 2, 1, lower =  0) # Truncated Normal hyperprior for mean 'd'
        mu_g = pm.TruncatedNormal('mu_g', 4, 1, lower =  1) # Truncated Normal hyperprior for mean 'g'
        
        
        # Define priors for parameters of the 5PL regression
        a = pm.Beta('a',              a1, a2,                         shape = n_sub) # Beta distribution for lower asymptote
        b = pm.TruncatedNormal('b', mu_b,  1,  lower = .5,            shape = n_sub) # Truncated Normal distribution for the slope
        c = pm.TruncatedNormal('c', mu_c,  1,  lower = .5, upper = 5, shape = n_sub) # Truncated Normal distribution for mid point
        d = pm.TruncatedNormal('d', mu_d,  1,  lower = .5,            shape = n_sub) # Truncated Normal distribution for upper asymptote
        g = pm.TruncatedNormal('g', mu_g,  1,  lower = .8,            shape = n_sub) # Truncated Normal distribution for asymmetry factor
             
        EN_slope = pm.HalfNormal('EN_slope', 1)
        Age_slope = pm.Normal('Age_slope', 0, 1)
        Inter_slope = pm.Normal('Inter_slope', 0, 1)
        
        s = b[idx] + (EN_slope * df_all['EmotionalNumbingZ'] +
                      Age_slope * df_all['AgeZ'] +
                      Inter_slope * df_all['EmotionalNumbingZ'] * df_all['AgeZ'])

        # Define prior for standard deviation of observations
        eps = pm.Exponential('eps', 1)

        # Define the 5PL regression model   
        y_hat = d[idx] + ((a[idx]-d[idx])/(1+(df_all['scale_rating'].values/c[idx])**s)**g[idx])

        # Define likelihood function using observed data
        rating = pm.Normal('rating',y_hat, eps, observed=df_all.rating)

        trace = pm.sample()#tune = 2000, draws = 2000, target_accept = 0.99)

    return trace

## Run models

In [8]:
# Assign a unique integer identifier to each unique ID in the dataset.
# The 'rank' method provides a ranking to each unique ID and 'dense' ensures there are no gaps between ranks.
# Subtracting 1 to make the ranking start from 0.
df_long_c['sub_id'] = df_long_c['ID'].rank(method='dense')-1

# Count the number of unique IDs (subjects)
n_subs = len(df_long_c['ID'].unique())

# Convert the 'sub_id' column to integer type as indices are typically integers.
idx = df_long_c['sub_id'].astype(int)


/tmp/ipykernel_555776/1484452449.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_long_c['sub_id'] = df_long_c['ID'].rank(method='dense')-1


In [9]:
trace_en = pl5_hyper_emotion(df_long_c, idx, n_subs)
trace_age = pl5_hyper_age(df_long_c, idx, n_subs)
trace_age_en = pl5_hyper_age_en(df_long_c, idx, n_subs)
trace_inter = pl5_hyper_inter(df_long_c, idx, n_subs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a1, a2, mu_b, mu_c, mu_d, mu_g, a, b, c, d, g, EN_slope, eps]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 3439 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a1, a2, mu_b, mu_c, mu_d, mu_g, a, b, c, d, g, Age_slope, eps]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 2734 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a1, a2, mu_b, mu_c, mu_d, mu_g, a, b, c, d, g, EN_slope, Age_slope, eps]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 2752 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a1, a2, mu_b, mu_c, mu_d, mu_g, a, b, c, d, g, EN_slope, Age_slope, Inter_slope, eps]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 2864 seconds.


In [10]:
compare_dict_en = {
                   'Emotional numbing': trace_en,
                   'Age': trace_age,
                   'EN and Age': trace_age_en,
                   'Interaction': trace_inter
                  }

comp_en = az.compare(compare_dict_en)
comp_en

/home/nachshon/anaconda3/envs/en/lib/python3.9/site-packages/arviz/stats/stats.py:812: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
/home/nachshon/anaconda3/envs/en/lib/python3.9/site-packages/arviz/stats/stats.py:812: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
/home/nachshon/anaconda3/envs/en/lib/python3.9

,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
Emotional numbing,0,19412.524677,2544.023964,0.000000,1.000000e+00,196.890971,0.000000,True,log
EN and Age,1,19409.709137,2547.311225,2.815539,0.000000e+00,196.881765,1.661881,True,log
Interaction,2,19405.891166,2551.005019,6.633510,0.000000e+00,196.914220,1.690268,True,log
Age,3,19404.898242,2551.400649,7.626434,9.769963e-14,196.966741,2.171544,True,log


In [11]:
az.summary(trace_en, var_names=['EN_slope'], hdi_prob=.89)

,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
EN_slope,0.1,0.031,0.051,0.15,0.001,0.0,1964.0,2761.0,1.0


In [12]:
az.summary(trace_inter, var_names=['EN_slope', 'Age_slope', 'Inter_slope'], hdi_prob=.89)

,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
EN_slope,0.108,0.032,0.056,0.159,0.001,0.001,2031.0,2547.0,1.0
Age_slope,0.031,0.033,-0.022,0.083,0.001,0.001,2106.0,3076.0,1.0
Inter_slope,-0.035,0.033,-0.085,0.016,0.001,0.000,2295.0,2928.0,1.0


In [13]:
def pl5_hyper(df_all, idx, n_sub):
    

    with pm.Model() as five_PL:
        
        # hyper priors
        # Define hyperpriors for group-level parameters
        a1   = pm.TruncatedNormal('a1',   2, 1, lower =  1) # Truncated Normal hyperprior for 'a1'
        a2   = pm.Normal('a2',           10, 1            ) # Normal hyperprior for 'a2'
        mu_b = pm.TruncatedNormal('mu_b', 2, 1, lower =  1) # Truncated Normal hyperprior for mean 'b'
        mu_c = pm.TruncatedNormal('mu_c', 2, 1, lower = .5) # Truncated Normal hyperprior for mean 'c'
        mu_d = pm.TruncatedNormal('mu_d', 2, 1, lower =  0) # Truncated Normal hyperprior for mean 'd'
        mu_g = pm.TruncatedNormal('mu_g', 4, 1, lower =  1) # Truncated Normal hyperprior for mean 'g'
        
        
        # Define priors for parameters of the 5PL regression
        a = pm.Beta('a',              a1, a2,                         shape = n_sub) # Beta distribution for lower asymptote
        b = pm.TruncatedNormal('b', mu_b,  1,  lower = .5,            shape = n_sub) # Truncated Normal distribution for the slope
        c = pm.TruncatedNormal('c', mu_c,  1,  lower = .5, upper = 5, shape = n_sub) # Truncated Normal distribution for mid point
        d = pm.TruncatedNormal('d', mu_d,  1,  lower = .5,            shape = n_sub) # Truncated Normal distribution for upper asymptote
        g = pm.TruncatedNormal('g', mu_g,  1,  lower = .8,            shape = n_sub) # Truncated Normal distribution for asymmetry factor

        # Define prior for standard deviation of observations
        eps = pm.Gamma('eps', 1, 1)

        # Define the 5PL regression model   
        y_hat = d[idx] + ((a[idx]-d[idx])/(1+(df_all['scale_rating'].values/c[idx])**b[idx])**g[idx])

        # Define likelihood function using observed data
        rating = pm.Normal('rating',y_hat, eps, observed=df_all.rating)

        # Sample from the posterior distribution
        trace = pm.sample()#tune = 3000, draws = 2000, target_accept = 0.99)

    return trace

In [14]:
trace = pl5_hyper(df_long_c, idx, n_subs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a1, a2, mu_b, mu_c, mu_d, mu_g, a, b, c, d, g, eps]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 2329 seconds.


In [15]:
compare_dict_en = {
                   'No regressors': trace,
                   'Emotional numbing': trace_en,
                  }

comp_en = az.compare(compare_dict_en)
comp_en

/home/nachshon/anaconda3/envs/en/lib/python3.9/site-packages/arviz/stats/stats.py:812: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
/home/nachshon/anaconda3/envs/en/lib/python3.9/site-packages/arviz/stats/stats.py:812: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
Emotional numbing,0,19412.524677,2544.023964,0.000000,1.0,196.890971,0.000000,True,log
No regressors,1,19406.988886,2550.063007,5.535791,0.0,196.985875,2.173637,True,log


In [1]:
trace_en.to_netcdf("../traces/trace_en1.nc")


NameError: name 'trace_en' is not defined